In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = "order_id long , order_date string, customer_id long,order_status string"

In [3]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [4]:
customer_schema = " customerid long , customer_fname string , customer_lname string , user_name string,password string , address string, city string, state string, pincode long "

In [5]:
customers_df = spark.read \
.format("csv") \
.schema(customer_schema) \
.load("/public/trendytech/retail_db/customers")

In [6]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '-1')

In [7]:
spark.sql("create database itv006277_optimizeJoin")

AnalysisException: org.apache.hadoop.hive.metastore.api.AlreadyExistsException: Database itv006277_optimizejoin already exists

In [ ]:
spark.sql("use itv006277_optimizeJoin")

In [ ]:
orders_df.write \
.mode("overwrite") \
.bucketBy(8, "customer_id") \
.sortBy("customer_id") \
.option("path", "orders201") \
.saveAsTable("orderstab201")

In [ ]:
spark.sql("describe formatted orderstab201").show(50, False)

In [ ]:
customers_df.write \
.mode("overwrite") \
.bucketBy(8, "customerid") \
.sortBy("customerid") \
.option("path", "customers201") \
.saveAsTable("customerstab201")

In [ ]:
spark.sql("select * from orderstab201 inner join customerstab201 on orderstab201.customer_id == customerstab201.customerid").write.format("noop").mode("overwrite").save()

In [8]:
spark.sql(" use itv006277_optimizeJoin")

""


In [9]:
spark.sql("show tables")

database,tableName,isTemporary
itv006277_optimiz...,customerstab201,false
itv006277_optimiz...,orderstab201,false


In [ ]:
spark.sql(" drop table itv006277_optimizeJoin.customerstab201")